<a href="https://colab.research.google.com/github/ElenaGordienko1/NLP/blob/main/NLP3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Выпонение лабораторной работы 3

## 1 часть

Импорт библиотек ⭐

In [73]:
import numpy as np
import pandas as pd

### Функция линейной регрессии

In [74]:
def linear_regression(X: np.ndarray) -> float:

    weight = np.random.randn()
    bias = np.random.randn()

    return float(X * weight + bias)

### Функция активации: сигмоида





In [75]:
def activation_func(x: float) -> float:

    return 1 / (1 + np.exp(-x))

### Нейрон



In [76]:
def neuron(x: np.ndarray):

    temp_result = linear_regression(x)
    result = activation_func(temp_result)
    return result


### Применение на датасете

In [77]:
df = pd.read_csv("/content/Mall_Customers.csv")

X = df['Annual Income (k$)'].values
Y = df['Spending Score (1-100)'].values

x_sample = X[0]

output = neuron(x_sample)
print(f"Входное значение X: {x_sample}, Выход нейрона: {output}")

Входное значение X: 15, Выход нейрона: 0.017129517486882132


## 2 часть

### Импорт, активация, нормализация

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pandas as pd

def activation_func(x):
    return 1 / (1 + np.exp(-x))

def activation_derivative(x):
    return x * (1 - x)

def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))


### Класс с возможностью задать количество нейронов слоя и произвести обучение

In [79]:
class NeuralNetwork:
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size, learning_rate=0.01):
        np.random.seed(42)
        self.lr = learning_rate

        self.W1 = np.random.randn(input_size, hidden1_size) * 0.1
        self.b1 = np.zeros((1, hidden1_size))
        self.W2 = np.random.randn(hidden1_size, hidden2_size) * 0.1
        self.b2 = np.zeros((1, hidden2_size))
        self.W3 = np.random.randn(hidden2_size, output_size) * 0.1
        self.b3 = np.zeros((1, output_size))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = activation_func(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = activation_func(self.z2)
        self.z3 = np.dot(self.a2, self.W3) + self.b3
        self.a3 = activation_func(self.z3)
        return self.a3

    def backward(self, X, y):
        m = X.shape[0]

        dZ3 = self.a3 - y
        dW3 = np.dot(self.a2.T, dZ3) / m
        db3 = np.sum(dZ3, axis=0, keepdims=True) / m

        dZ2 = np.dot(dZ3, self.W3.T) * activation_derivative(self.a2)
        dW2 = np.dot(self.a1.T, dZ2) / m
        db2 = np.sum(dZ2, axis=0, keepdims=True) / m

        dZ1 = np.dot(dZ2, self.W2.T) * activation_derivative(self.a1)
        dW1 = np.dot(X.T, dZ1) / m
        db1 = np.sum(dZ1, axis=0, keepdims=True) / m

        self.W3 -= self.lr * dW3
        self.b3 -= self.lr * db3
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.forward(X)
            self.backward(X, y)
            if epoch % 100 == 0:
                loss = np.mean((y - self.a3) ** 2)
                print(f'Epoch {epoch}, Loss: {loss:.4f}')


### Проверка



In [80]:
df = pd.read_csv("/content/Mall_Customers.csv")

X_raw = df['Annual Income (k$)'].values.reshape(-1, 1)
Y_raw = df['Spending Score (1-100)'].values.reshape(-1, 1)

X_normalized = normalize(X_raw)
Y_normalized = normalize(Y_raw)

input_size = 1
hidden1_size = 5
hidden2_size = 5
output_size = 1
learning_rate = 0.01
epochs = 1000

nn = NeuralNetwork(input_size, hidden1_size, hidden2_size, output_size, learning_rate)
nn.train(X_normalized, Y_normalized, epochs)

predictions = nn.forward(X_normalized)

print("Настоящее значение:", Y_raw.flatten()[:5])
print("Предсказание:", (predictions * 100).flatten()[:5])

Epoch 0, Loss: 0.0692
Epoch 100, Loss: 0.0691
Epoch 200, Loss: 0.0691
Epoch 300, Loss: 0.0691
Epoch 400, Loss: 0.0691
Epoch 500, Loss: 0.0691
Epoch 600, Loss: 0.0691
Epoch 700, Loss: 0.0691
Epoch 800, Loss: 0.0691
Epoch 900, Loss: 0.0691
Настоящее значение: [39 81  6 77 40]
Предсказание: [50.20817834 50.20817834 50.20819002 50.20819002 50.2082017 ]


### Применение

## 3 часть

### Реализация GPT

### Обработка текста

In [88]:
def tokenize_text(text):
    return text.split()

def create_input_target_sequences(sequences):
    input_sequences, target_sequences = [], []
    for sentence in sequences:
        mid = len(sentence) // 2
        input_sequences.append(np.array(sentence[:mid]))
        target_sequences.append(np.array(sentence[mid:]))
    return input_sequences, target_sequences

embedding_dim = 128
attention_dim = 32
seq_len = 64
drop_rate = 0.1

text = """вы помните,
вы всё, конечно, помните,
как я стоял,
приблизившись к стене,
взволнованно ходили вы по комнате
и что-то резкое
в лицо бросали мне.
вы говорили:
нам пора расстаться,
что вас измучила
моя шальная жизнь,
что вам пора за дело приниматься,
а мой удел — катиться дальше, вниз.
"""

sentences = text.strip().replace("\n", " ").split(". ")
words = set(" ".join(sentences).split())

word_to_index = {word: idx for idx, word in enumerate(words)}
index_to_word = {idx: word for word, idx in word_to_index.items()}
vocab_size = len(word_to_index)

sequences = [[word_to_index[word] for word in sentence.split()] for sentence in sentences]
input_sequences, target_sequences = create_input_target_sequences(sequences)

input_sequences = [seq.reshape(1, len(seq), 1) for seq in input_sequences]
target_sequences = [seq.reshape(1, len(seq), 1) for seq in target_sequences]

### Класс голова

In [89]:
class Head:
    def __init__(self, embedding_dim, attention_dim, seq_len, drop_rate, vocab_size):
        self.embedding_dim = embedding_dim
        self.attention_dim = attention_dim
        self.seq_len = seq_len
        self.drop_rate = drop_rate
        self.vocab_size = vocab_size

        self.key_matrix = np.random.randn(embedding_dim, attention_dim) * 0.1
        self.query_matrix = np.random.randn(embedding_dim, attention_dim) * 0.1
        self.value_matrix = np.random.randn(embedding_dim, attention_dim) * 0.1
        self.output_projection = np.random.randn(attention_dim, embedding_dim) * 0.1
        self.mask_matrix = np.tril(np.ones((seq_len, seq_len)))

        self.embeddings = np.random.randn(vocab_size, embedding_dim) * 0.1

    def compute(self, input_tensor):
        B, T, C = input_tensor.shape

        key_proj = np.dot(input_tensor, self.key_matrix)     # (B, T, hs)
        query_proj = np.dot(input_tensor, self.query_matrix) # (B, T, hs)
        value_proj = np.dot(input_tensor, self.value_matrix) # (B, T, hs)

        scores = np.matmul(query_proj, key_proj.transpose(0, 2, 1)) * (self.attention_dim ** -0.5)
        scores = np.where(self.mask_matrix[:T, :T] == 0, float('-inf'), scores)

        exp_scores = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
        attention_weights = exp_scores / np.sum(exp_scores, axis=-1, keepdims=True)

        # Dropout
        dropout_mask = np.random.rand(*attention_weights.shape) > self.drop_rate
        attention_weights *= dropout_mask

        context = np.matmul(attention_weights, value_proj)   # (B, T, hs)
        output_tensor = np.dot(context, self.output_projection)  # (B, T, C)

        return output_tensor, context

    def forward(self, input_tensor):
        output_tensor, _ = self.compute(input_tensor)
        return output_tensor

    def backward(self, input_tensor, grad_output):
        return grad_output

    def train(self, input_tensor, target_tensor, learning_rate=0.001):
        output_tensor, context = self.compute(input_tensor)
        B, T, C = output_tensor.shape

        target_embeddings = self.embeddings[target_tensor.squeeze(axis=-1)]  # (B, T, C)
        loss = np.mean(np.sum((output_tensor - target_embeddings) ** 2, axis=-1))

        grad_output = (output_tensor - target_embeddings) / target_embeddings.size  # (B, T, C)

        grad_output_projection = np.dot(context.reshape(-1, self.attention_dim).T,
                                        grad_output.reshape(-1, self.embedding_dim))
        self.output_projection -= learning_rate * grad_output_projection

        mean_grad = np.mean(grad_output, axis=0)  # (T, C)
        mean_input = np.mean(input_tensor, axis=0)  # (T, C)

        grad_key = np.dot(mean_input.T, mean_grad)
        grad_query = np.dot(mean_input.T, mean_grad)
        grad_value = np.dot(mean_input.T, mean_grad)

        self.key_matrix -= learning_rate * grad_key[:, :self.attention_dim]
        self.query_matrix -= learning_rate * grad_query[:, :self.attention_dim]
        self.value_matrix -= learning_rate * grad_value[:, :self.attention_dim]

        return loss

    def print_embeddings(self, epoch, input_tensor, output_tensor):
        print(f"Epoch {epoch} - Input[0][:5]: {input_tensor[0, 0, :5]}")
        print(f"Epoch {epoch} - Target[0][:5]: {output_tensor[0, 0, :5]}")

### Инициализация модели и обучение

In [98]:

input_sequences = [np.random.randn(seq.shape[0], seq.shape[1], embedding_dim) for seq in input_sequences]

attention_head = Head(embedding_dim, attention_dim, seq_len, drop_rate, vocab_size)

epochs = 10
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_sequences)):
        loss = attention_head.train(input_sequences[i], target_sequences[i], learning_rate=0.01)
        total_loss += loss

        attention_head.print_embeddings(epoch + 1, input_sequences[i], target_sequences[i])

        output = attention_head.forward(input_sequences[i])  # (B, T, C)
        predicted_embedding = output[0, 0]  # вектор (C,)

        distances = np.linalg.norm(attention_head.embeddings - predicted_embedding, axis=1)
        predicted_idx = np.argmin(distances)
        predicted_word = index_to_word[predicted_idx]

        real_word_idx = target_sequences[i][0][0].item()
        real_word = index_to_word[real_word_idx]

        print(f"Предсказанное слово: {predicted_word}, Реальное слово: {real_word}")

    print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {total_loss / len(input_sequences)}")

Epoch 1 - Input[0][:5]: [ 1.66074572  0.4133736  -0.33491507 -0.7947416   0.10687018]
Epoch 1 - Target[0][:5]: [38]
Предсказанное слово: по, Реальное слово: взволнованно
Epoch 1 - Input[0][:5]: [-0.22496576  1.67604955 -1.20343388 -0.47065288 -1.41418608]
Epoch 1 - Target[0][:5]: [21]
Предсказанное слово: расстаться,, Реальное слово: вам
Epoch 1/10, Average Loss: 15.341797185155418
Epoch 2 - Input[0][:5]: [ 1.66074572  0.4133736  -0.33491507 -0.7947416   0.10687018]
Epoch 2 - Target[0][:5]: [38]
Предсказанное слово: дело, Реальное слово: взволнованно
Epoch 2 - Input[0][:5]: [-0.22496576  1.67604955 -1.20343388 -0.47065288 -1.41418608]
Epoch 2 - Target[0][:5]: [21]
Предсказанное слово: расстаться,, Реальное слово: вам
Epoch 2/10, Average Loss: 16.49956671150332
Epoch 3 - Input[0][:5]: [ 1.66074572  0.4133736  -0.33491507 -0.7947416   0.10687018]
Epoch 3 - Target[0][:5]: [38]
Предсказанное слово: по, Реальное слово: взволнованно
Epoch 3 - Input[0][:5]: [-0.22496576  1.67604955 -1.2034338